# Test purpose

Test precision of potential evaluation comparing direct with moment based

source points are located in a square around origin, and the moments are evaluated at the origin. Alternatively, they can be placed uniformly in a sphere. 'source_area_size' refers to the radius of this sphere - or for the box, the "half-size" of the box in each direction.

For the first and second test a fixed p is used - defined below. 

First test: 
- target points are first located in a "halo" around the origin.
- Same number of target and source points
- located between 10 and 11 times the 'source_area_size
- potential is calculated at target points both directly and using moments
- Relative error is shown

Second test:
- target points are located in a sphere at a distance 
- center is located at 10 times 'source_area_size'
- potential is calculated at target points both directly and using moments
- Relative error is shown

Third test:
- a selection p's and distances are chosen
- Potenential at each distances for each p is calculated using both direct and moment based
- error as function of p and distance is compared in selected plots. 

In [ ]:
import sys, os
from pathlib import Path
import matplotlib.pyplot as plt

# Find the repository root by searching upward for a 'pyFMM' directory
start = Path.cwd().resolve()
repo_root = None
for p in [start] + list(start.parents):
    if (p / 'pyFMM').is_dir():
        repo_root = str(p)
        break
if repo_root is None:
    raise RuntimeError("Could not find 'pyFMM' in any parent directory of cwd")
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
    
from pyFMM import *
import numpy as np

In [ ]:
np.random.seed(42)
N = 10000                              # number of points
q = np.random.choice([-1, 1], size=N)  # source strengths with magnitude 1 and random sign
#q = np.ones(N)                        # source strengths all positive 1
source_area_size = 1.0
#---------- moment order ---------
p = 4
#---------------------------------

In [ ]:
##--------- generate a number of random points inside sphere ----------
## NOTE - with this spherical symmetry, higher order moments are less important. 
#X = generate_points_in_sphere(N, source_area_size)
##------------------------------------------------------------------
##--------- convert to polar coordinates ----------------
#x_polar = cart_to_sphe(X)
##------------------------------------------------------

In [ ]:
#--------- generate a number of random points in side box ----------
# NOTE - with this we get slight asymmetries compared to spherical distribution
#       this meains higher order moments are more imporant
LLC = np.array([ -1.0, -1.0, -1.0 ]) * source_area_size   # Lower Left Corner
URC = np.array([  1.0,  1.0,  1.0 ]) * source_area_size   # Upper Right Corner
X = np.random.rand(N, 3)             # uniform in [0,1)
X = LLC + (URC - LLC) * X            # uniform in [LLC, URC]
#------------------------------------------------------------------
#--------- convert to polar coordinates ----------------
x_polar = cart_to_sphe(X)
#------------------------------------------------------

In [ ]:
#--------- generate target points outside box ----------
# Points lie in a "halo" around the box - roughly at a distance size(URC-LLC)*r_margin
Ntgt = N
r_min = source_area_size * 10
r_max = source_area_size * 11
x_tgt = generate_points_in_sphere(Ntgt, r_max, r_min=r_min)
x_tgt_polar = cart_to_sphe(x_tgt)
#------------------------------------------------------

In [ ]:
fig, ax = plot_points_3d(
    X, x_tgt,
    labels=['Source Points', 'Target Points'],
    colors=['blue', 'red'],
    sizes=[0.5, 0.5],
    title='3D Source and Target Distribution',
)
#------- optional plotting with interactive plotly -------
# fig = plot_points_3d_plotly(
#     X, x_tgt,
#     labels=["Source Points", "Target Points"],
#     sizes=[2,2],
#     title="3D Source and Target Points"
# )
# fig.show() 
#------------------------------------------------------

In [ ]:
#----------  compute moments and potentials ----------
M_std = P2M(x_polar, q, p=p)
P_mom = P_moment(M_std, x_tgt_polar)
P_dir_vec =  P_direct_cart(X, q, x_tgt, eps=1e-10)
#------------------------------------------------------

In [ ]:
#--------- comput relative error between direct summation and moment evaluation ----------
relative_errors = np.abs(P_dir_vec - P_mom) / np.abs(P_dir_vec)
#------------------------------------------------------
#---------- plot relative errors ----------
fig, ax = plot_ylog(
    relative_errors,
    title='Relative Error per Target',
    xlabel='Target Index',
    ylabel='Relative Error (log scale)',
    sizes=[2],
    legend=False
)
plt.show()
#------------------------------------------

In [ ]:
#--------- generate target points outside box ----------
# Points lie in a small cloud centered at a distance 'dist' from origin
Ntgt = N
dist = 10 * source_area_size
center = np.array([1.0, 1.0, 1.0]) * dist
r_max = source_area_size
x_tgt = generate_points_in_sphere(Ntgt, r_max, center=center)
x_tgt_polar = cart_to_sphe(x_tgt)
#------------------------------------------------------

In [ ]:
#----------------- plot source and target points -----------------
fig, ax = plot_points_3d(
    X, x_tgt,
    labels=['Source Points', 'Target Points'],
    colors=['blue', 'red'],
    sizes=[0.5, 0.5],
    title='3D Source and Target Distribution',
)
#------------------------------------------------------------------

In [ ]:
#---------- compute moments and potentials ----------
M_std = P2M(x_polar, q, p=p)
P_mom = P_moment(M_std, x_tgt_polar)
P_dir_vec =  P_direct_cart(X, q, x_tgt, eps=1e-10)
#------------------------------------------------------

In [ ]:
#--------- comput relative error between direct summation and moment evaluation ----------
relative_errors = np.abs(P_dir_vec - P_mom) / np.abs(P_dir_vec)
#------------------------------------------------------
fig, ax = plot_ylog(
    relative_errors,
    title='Relative Error per Target',
    xlabel='Target Index',
    ylabel='Relative Error (log scale)',
    sizes=[2],
    legend=False
)
plt.show()

In [ ]:
#--------------- evaluate potential at center point --------------
center = np.array([1.0, 1.0, 1.0]) * dist 
center_sphe = cart_to_sphe(np.array([center]))[0]
P_center_mom = P_moment(M_std, np.array([center_sphe]))[0]
P_center_dir =  P_direct_cart(X, q, np.array([center]), eps=1e-10)[0]
#------------------------------------------------------
#-------------- print results --------------
print("Moment eval at center:", P_center_dir)
print("Direct eval at center:", P_center_mom)
print("Diff = ", np.abs(P_center_dir - P_center_mom))
#------------------------------------------------------

In [ ]:
#------------- compute theoretical error bound at center point --------------
A = np.sum(np.abs(q))
r = np.linalg.norm(center)
a = source_area_size
err_bound = (A / (r -a )) * ( a / r)**(p+1)
rel_error_bound = err_bound / np.abs(P_center_dir)
print("Theoretical error bound:", err_bound)
print("Theorerical relative error bound", rel_error_bound)
#----------------------------------------------------------------------------
#-------------- compare to actual center error ------------------------
actual_center_error = np.abs(P_center_dir - P_center_mom)
print("")
print("Actual center error:", actual_center_error , " relative to bound", actual_center_error / err_bound * 100, "%")
print("")
#----------------------------------------------------------------------
#------------- compare to actual errors over all targets ----------------
print("Actual error max:", np.max(relative_errors)  , " relative to bound",  np.max(relative_errors)/rel_error_bound * 100, "%")
print("Actual error min:", np.min(relative_errors)  , " relative to bound",  np.min(relative_errors)/rel_error_bound * 100, "%")
print("Actual error avg:", np.mean(relative_errors) , " relative to bound",  np.mean(relative_errors)/rel_error_bound * 100, "%")
#-----------------------------------------------------------------------

In [ ]:
#--------------- create list of p order and separation distances --------------
all_p = np.arange(1, 10)            
sep_dist = np.arange(2,16,2) * source_area_size
#-------------------------------------------------------------------------------
#---------- allocate arrays for results ----------
P_mom = np.zeros((len(all_p), len(sep_dist)))
P_dir = np.zeros((len(all_p), len(sep_dist)))
#------------------------------------------------
#------------- loop over p and separation distances ----------
for ip, p in enumerate(all_p):
    #------------- compute moments for given p ----------
    M_std = P2M(x_polar, q, p=p)  
    #----------------------------------------------------
    for jd, dist in enumerate(sep_dist):
        #------------- define center point at given distance ----------
        center = np.array([1.0, 1.0, 1.0]) * dist
        center_sphe = cart_to_sphe(np.array([center]))[0]
        #--------------------------------------------------------------
        #--------------- compute potentials at center point --------------
        P_mom[ip, jd] = P_moment(M_std, np.array([center_sphe]))[0]
        P_dir[ip, jd] = P_direct_cart(X, q, np.array([center]), eps=1e-10)[0]
        #----------------------------------------------------------------
#----------------------------------------------------------
#----------- compute errors ----------------
abs_err = np.abs(P_mom - P_dir)
with np.errstate(divide='ignore', invalid='ignore'):
    rel_err = abs_err / np.maximum(np.abs(P_dir), 1e-300)  # robust denominator
#--------------------------------------------


In [ ]:
#---------- first plot to verify parity --------------
plt.figure(figsize=(8,6))
plt.scatter(P_dir.ravel(), P_mom.ravel(), s=12, alpha=0.6)
lims = [np.nanmin(P_dir), np.nanmax(P_dir)]
plt.plot(lims, lims, lw=1)  # y=x
plt.xlabel("Direct potential")
plt.ylabel("Multipole potential")
plt.title("Parity: P_mom vs P_dir")
plt.tight_layout()
plt.show()


# --- 1) Relative error vs p, one curve per separation ---
plt.figure(figsize=(7,5))
for j, R in enumerate(sep_dist):
    plt.plot(list(all_p), rel_err[:, j], marker='o', label=f'R={R}')
plt.yscale('log')
plt.xlabel('Expansion order p')
plt.ylabel('Relative error')
plt.title('Relative error vs p (per separation)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()
plt.tight_layout()

# --- 2) Relative error vs separation, one curve per p ---
plt.figure(figsize=(7,5))
for i, p in enumerate(all_p):
    plt.plot(sep_dist, rel_err[i, :], marker='o', label=f'p={p}')
plt.xscale('log', base=2)     # you used R = 2**i; base-2 makes spacing intuitive
plt.yscale('log')
plt.xlabel('Separation R (|center|)')
plt.ylabel('Relative error')
plt.title('Relative error vs separation (per p)')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(ncol=2)
plt.tight_layout()

# --- 3) Heatmap of log10 relative error over (p, R) ---
plt.figure(figsize=(6.5,5))
im = plt.imshow(
    np.log10(rel_err),          # rows = p, cols = R
    origin='lower',
    aspect='auto',
    extent=[np.log2(sep_dist[0]), np.log2(sep_dist[-1]), all_p[0], all_p[-1]]
)
cbar = plt.colorbar(im, label=r'$\log_{10}(\mathrm{relative\ error})$')
plt.xlabel(r'$\log_2 R$')
plt.ylabel('Expansion order p')
plt.title('Error heatmap over (p, R)')
plt.tight_layout()

# --- Optional: magnitude/phase sanity plots (if complex) ---
if np.iscomplexobj(P_dir):
    phase_diff = np.angle(P_mom) - np.angle(P_dir)
    phase_diff = (phase_diff + np.pi) % (2*np.pi) - np.pi  # wrap to [-pi, pi]
    plt.figure(figsize=(7,4))
    for j, R in enumerate(sep_dist):
        plt.plot(list(all_p), np.abs(phase_diff[:, j]), marker='o', label=f'R={R}')
    plt.yscale('log')
    plt.xlabel('p'); plt.ylabel('abs phase diff [rad]')
    plt.title('Phase error vs p')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.legend()
    plt.tight_layout()

plt.show()
